In [ ]:
### Some generic imports that will be used throughout the workshop
import datetime as dt
import time
from pathlib import Path

In [ ]:
import requests

API_BASE = "https://api.irail.be"
UA = "irail-liveboard-swiftbar/1.1 (contact: test@riskconcile.com)"

# --- Basic config ---
# Either set STATION_ID (preferred) or STATION name
STATION_ORIGIN_ID = "BE.NMBS.008833001" # Leuven
STATION_ORIGIN = "Leuven"
STATION_DEST_ID = "BE.NMBS.008813003" # Brussels-Central
STATION_DEST = "Brussels-Central"
ARRDEP = "departure"
LANG = "en"
MAX_ROWS = 10

def _get(path, params):
    headers = {"User-Agent": UA}
    params = dict({"format": "json", "lang": LANG}, **params)
    r = requests.get(f"{API_BASE}{path}", params=params, headers=headers, timeout=15)
    r.raise_for_status()
    return r.json()

def get_liveboard():
    params = {"arrdep": ARRDEP, "alerts": "false", "id": STATION_ORIGIN_ID}
    return _get("/liveboard/", params)

def fmt_time(ts):
    try:
        return dt.datetime.fromtimestamp(int(ts)).strftime("%H:%M")
    except Exception:
        return "??:??"

def fmt_delay(delay_seconds):
    try:
        d = int(delay_seconds)
        return "" if d <= 0 else f"+{d//60}′"
    except Exception:
        return ""

# ------- Filter helper (fetch vehicle -> check stops) -------
def passes_by(vehicle_id, target):
    if not vehicle_id:
        return False
    # Endpoint returns the sequence of stops for the vehicle
    data = _get("/vehicle/", {"id": vehicle_id})
    stops = data.get("stops", {}).get("stop", [])
    if isinstance(stops, dict):  # normalize single stop edge case
        stops = [stops]
    idx = [stop.get("station", {}) for stop in stops].index(STATION_ORIGIN)
    for st in stops[idx:]:
        name = st.get("station", {})
        if name.lower() == target.lower():
            return True
    return False
# -----------------------------------------------------------------

def main():
    try:
        data = get_liveboard()
    except Exception as e:
        print("🚂 iRail: error"); print("---"); print(str(e)); return

    key = "departures" if ARRDEP == "departure" else "arrivals"
    rows = (data.get(key, {}) or {}).get(key[:-1], [])
    if isinstance(rows, dict):
        rows = [rows]

    # ------- NEW: apply via filter using vehicle endpoint -------
    filtered = []
    for item in rows:
        vehicle_id = item.get("vehicle")
        try:
            if passes_by(vehicle_id, STATION_DEST):
                filtered.append(item)
        except Exception:
            # If vehicle lookup fails, just skip that item
            pass
        if len(filtered) >= MAX_ROWS:
            break
    rows = filtered
    # ------------------------------------------------------------
    if len(rows) > 0:
        print(f"🚂 Next Departure: {fmt_time(rows[0].get('time'))}")
        print("---")
    else:
        print("No matching trains (via Brussels-Central)."); return

    # Render lines
    href = "https://irail.be/"
    for item in rows:
        ts = item.get("time")
        when = fmt_time(ts) if ts else "??:??"
        dest = item.get("station", "?")
        platform = (item.get("platform", {}) or {}).get("name") if isinstance(item.get("platform"), dict) else item.get("platform") or "?"
        delay = fmt_delay(item.get("delay", 0))
        canceled = str(item.get("canceled", "0")) in ("1", "true", "True")
        status = "❌" if canceled else ("⏱" if delay else "•")
        print(f"{status} {when}  {dest}  (pf {platform}) {delay} | href={href}")

    print("---")
    print("Refresh now ↻ | refresh=true")

if __name__ == "__main__":
    main()


In [ ]:
### We will use watchdog to monitor the directory for new files
from watchdog.events import FileSystemEventHandler, FileCreatedEvent
from watchdog.observers import Observer

### We will use custom utility functions to extract invoice data from pdfs and handle
### different file operations. Since they are not the key focus of this workshop, we
### will not go into their implementation details. But if you are curious, feel free
### to check out the code in the utils/ directory.
from utils.invoice_data_extractor import extract_invoice_data
from utils.file_handling import append_data_to_excel_file, wait_until_file_is_ready

### Let's define the directory to watch and the file path to the invoice data excel 
### overview.
WATCHED_DIR = Path("invoices")
INVOICE_DATA_FILE_PATH = WATCHED_DIR / "invoice_data.xlsx"

### Watchdog lets us define custom functionality for different file system events. File
### system events include file creation, modification, deletion, and movement. Here, we
### will define a custom event handler that reacts to new file creation events.
class InvoiceFileHandler(FileSystemEventHandler):
    def on_created(self, event):
        if not isinstance(event, FileCreatedEvent):
            return
        
        p = Path(event.src_path)
        
        if p.suffix.lower() == ".pdf":
            wait_until_file_is_ready(p)
            print(f"New invoice detected: {p.name}")
            extracted_data = extract_invoice_data(p)
            append_data_to_excel_file(extracted_data, INVOICE_DATA_FILE_PATH)

### Let's define a function to start watching the directory for new files, and act 
### when the defined file system events occur. We will therefore have to use the 
### custom event handler we defined above, as well as watchdog's own observer.
def start_watching():
    WATCHED_DIR.mkdir(exist_ok=True)
    observer = Observer()
    observer.schedule(InvoiceFileHandler(), WATCHED_DIR, recursive=False)
    observer.start()
    print(f"Watching directory: {WATCHED_DIR}")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

### Finally, let's start the file watching process and see it in action.
start_watching()